# Read Contrucor JSON File to Spark Dataframe reader

In [0]:
client_id=dbutils.secrets.get(scope="formula1-scope",key="client-id")
tenant_id=dbutils.secrets.get(scope="formula1-scope",key="tenant-id")
client_secret=dbutils.secrets.get(scope="formula1-scope",key="client-secret")

spark.conf.set("fs.azure.account.auth.type.databrickspracticesa.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.databrickspracticesa.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.databrickspracticesa.dfs.core.windows.net", client_id)
spark.conf.set("fs.azure.account.oauth2.client.secret.databrickspracticesa.dfs.core.windows.net", client_secret)
spark.conf.set("fs.azure.account.oauth2.client.endpoint.databrickspracticesa.dfs.core.windows.net",f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")

In [0]:
display(dbutils.fs.ls("abfss://raw@databrickspracticesa.dfs.core.windows.net"))

In [0]:
#Construc Schema
from pyspark.sql.types import StructType,StructField,IntegerType,StringType
constructors_schema="constructorId integer,constructorRef string,name string,nationality string,url string"

#Read Constructors JSON Data
constructor_df=spark.read\
.schema(constructors_schema).json("abfss://raw@databrickspracticesa.dfs.core.windows.net/constructors.json")
display(constructor_df.limit(10))



In [0]:
constructor_df.printSchema()


In [0]:
from pyspark.sql.functions import lit;
from pyspark.sql.functions import to_timestamp
from pyspark.sql.functions import concat
from pyspark.sql.functions import current_timestamp

In [0]:
constructor_select_df=constructor_df.select(constructor_df["constructorId"],constructor_df["constructorRef"],constructor_df["name"],constructor_df["nationality"])
display(constructor_select_df.limit(10))
constructor_renamed_df=constructor_select_df.withColumnRenamed("constructorId","constructor_id")\
.withColumnRenamed("constructorRef","constructor_ref")\
.withColumn("ingestion_date",current_timestamp()).withColumn("env",lit("itg"))
display(constructor_renamed_df.limit(10))


# Write data to parquet

In [0]:
#Write with Partition
constructor_renamed_df.write.mode("overwrite").parquet("abfss://processed@databrickspracticesa.dfs.core.windows.net/constructors")

In [0]:
#Display the data
display(spark.read.parquet("abfss://processed@databrickspracticesa.dfs.core.windows.net/constructors"))